In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-9b-it", # instruction
    max_seq_length = max_seq_length,
    dtype = dtype,
)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
device = "cuda:0"

query = ''' ترجم الي اللهجه المصريه: "After graduation I came back to Egypt to study in college, and now I have been living in Egypt for over 5 years and 3 months, I love Egypt."'''


prompt_template = """<start_of_turn>user يوجد أدناه تعليمات تصف المهمة. اكتب الرد الذي يكمل الطلب بشكل مناسب.\n\n{query}<end_of_turn>\n<start_of_turn>model"""

prompt = prompt_template.format(query=query)

inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

model_inputs = inputs.to(device)

In [ ]:
from transformers import TextStreamer


text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)